In [1]:
import os
os.chdir('..')

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%load_ext autoreload
%autoreload 2

In [6]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)
import plotly
import plotly.express as px
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import src.constants as const

In [14]:
import pandas as pd

In [48]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))
df = dataset.dataframe

In [49]:
df[(df.mario_speed<df.mario_speed.quantile(.2)) & (df.pipe_x > df.pipe_x.quantile(.8))]

,Unnamed: 0,imgs_folder_path,box_x,pipe_x,enemy_speed,mario_speed,answer_mario_box,answer_enemy_pipe,answer_mario_enemy
88,88,data/imgs_series/00089,-1.399868,1.810335,-1.514621,-1.514371,-1.079707,2.601959,-0.270603


In [50]:
df[(df.mario_speed>df.mario_speed.quantile(.8)) & (df.enemy_speed < df.enemy_speed.quantile(.2))]

,Unnamed: 0,imgs_folder_path,box_x,pipe_x,enemy_speed,mario_speed,answer_mario_box,answer_enemy_pipe,answer_mario_enemy
79,79,data/imgs_series/00080,-1.329385,0.998768,-1.213204,1.682634,-1.481927,1.655048,-0.614671


In [51]:
df[(df.enemy_speed>df.enemy_speed.quantile(.8)) & (df.pipe_x < df.pipe_x.quantile(.2))]

,Unnamed: 0,imgs_folder_path,box_x,pipe_x,enemy_speed,mario_speed,answer_mario_box,answer_enemy_pipe,answer_mario_enemy
29,29,data/imgs_series/00030,1.109336,-1.021131,1.198133,-0.504790,1.214727,-1.279393,0.978450
74,74,data/imgs_series/00075,0.862644,-1.147375,1.047425,0.673054,0.512896,-1.234699,-0.076306


In [52]:
def plot_first_and_last_pic(index1, index2):
    dir_path = f'data/imgs_series/{index1:05d}'
    files = os.listdir(dir_path)
    files.sort()
    img_ls = [open(f'{dir_path}/{f}', 'rb').read() for f in files]
    wi_ls = [widgets.Image(value=img, format='png', width=200) for img in img_ls]
    sidebyside = widgets.HBox(wi_ls[::2])
    print(f'mario speed: {df[df.imgs_folder_path == dir_path].mario_speed.values[0]}'
          f', enemy speed: {df[df.imgs_folder_path == dir_path].enemy_speed.values[0]}',
          f', box x: {df[df.imgs_folder_path == dir_path].box_x.values[0]}',
          f', pipe x: {df[df.imgs_folder_path == dir_path].pipe_x.values[0]}')
    display(sidebyside)
    
    dir_path = f'data/imgs_series/{index2:05d}'
    files = os.listdir(dir_path)
    files.sort()
    img_ls = [open(f'{dir_path}/{f}', 'rb').read() for f in files]
    wi_ls = [widgets.Image(value=img, format='png', width=200) for img in img_ls]
    sidebyside = widgets.HBox(wi_ls[::2])
    print(f'mario speed: {df[df.imgs_folder_path == dir_path].mario_speed.values[0]}'
          f', enemy speed: {df[df.imgs_folder_path == dir_path].enemy_speed.values[0]}',
          f', box x: {df[df.imgs_folder_path == dir_path].box_x.values[0]}',
          f', pipe x: {df[df.imgs_folder_path == dir_path].pipe_x.values[0]}')
    display(sidebyside)
    
interact(plot_first_and_last_pic, index1=list(range(1, len(df)-1)), index2=list(range(1, len(df)-1)))

interactive(children=(Dropdown(description='index1', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

<function __main__.plot_first_and_last_pic(index1, index2)>

In [ ]:
cols = ['box_x', 'pipe_x', 'enemy_speed', 'mario_speed']
fig = go.Figure()
fig = make_subplots(rows=1, cols=4)
for i, c in enumerate(const.HIDDEN_STATE_COLS):
    fig.add_trace(go.Histogram(x=df[c].values), row=1, col=i+1)
    fig.update_xaxes(title_text=c, row=1, col=i+1)

# Overlay both histograms
fig.update_layout(barmode='overlay', showlegend=False, title_text="Hidden state & question distributions")
# Reduce opacity to see both histograms
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
fig = go.Figure()
for c in const.ANSWER_COLS:
    fig.add_trace(go.Histogram(x=df[c].values, name=c))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_layout(title_text="Optimal answer distributions")
fig.update_xaxes(title_text="Optimal answer value")
fig.update_yaxes(title_text="Frequency")
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
import src.constants as const
def plot_answer(answer_col, col0, col1):
    fig = px.scatter_3d(df, x=col0, y=col1, z=answer_col, color=answer_col)
    fig.show()
interact(plot_answer, col0=const.HIDDEN_STATE_COLS, col1=const.HIDDEN_STATE_COLS,
         answer_col=const.ANSWER_COLS)